In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import time
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.keras.layers.core import Dense
from tensorflow.python.ops.gen_array_ops import shape
import random
import sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

## Model

In [ ]:
def Model(X_test):
    input = tf.keras.Input(shape=X_test[0].shape)
  # CNN
    x = tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu')(input)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling1D(3, padding='same')(x)
    x = tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling1D(3, padding='same')(x)
    x = tf.keras.layers.Flatten()(x)

    output = tf.keras.layers.Dense(64)(x)
    output = tf.keras.layers.Dense(2)(output)
  # output = tf.keras.layers.Dropout(.5,input_shape=(5,))(output)
    output = tf.keras.activations.sigmoid(output)
    model = tf.keras.Model(inputs=[input], outputs=[output])
    return model

# Hàm đánh giá

In [ ]:
def Evaluation_Function(global_model,X_test,y_test):
    y_predict = global_model.predict(X_test).argmax(1)
    y_true = np.array(y_test).argmax(1)
    acc = tf.keras.metrics.Accuracy()
    acc.update_state(y_true, y_predict)
    acc = acc.result().numpy()
    recall = tf.keras.metrics.Recall()
    recall.update_state(y_true, y_predict)
    recall = recall.result().numpy()
    precision = tf.keras.metrics.Precision()
    precision.update_state(y_true, y_predict)
    precision = precision.result().numpy()
    f_score =  2 * recall * precision / (recall + precision)
    return [acc,recall,precision,f_score]

#Load func

In [ ]:
def load_data(path, test = True,r = 1, agents = 1):
    if test:
        X = pd.read_csv(path)
    else:
        path = path + "SB19_CCFDUBL_BAL_P" + str(agents)+ "_2C" + ".csv"
        X = pd.read_csv(path)

    Y = X.iloc[:,-1:]
    num_records = Y.shape[0]
    X = tf.constant(X.values[:,0:30])
    X = tf.reshape(X, (num_records, 30, 1))
    Y = tf.one_hot(Y.values[:,0:], 2)
    Y = tf.reshape(Y, (num_records, 2))
    return X,Y

In [ ]:
path = "/content/drive/MyDrive/SwarmLearn/Fraunt/Dataset/SB19_CCFDUBL_BAL_TEST_2C.csv"
X_test, Y_test = load_data(path)

In [ ]:
from keras.optimizers import SGD
from sklearn.metrics import precision_score, recall_score, f1_score
model = Model(X_test)
optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.01)
model.compile(loss = 'binary_crossentropy', optimizer=optimizer, metrics=["accuracy"]
)

# Dữ liệu bình thường

In [104]:
def FL_Train(rounds,agents,global_model):
  for r in range(rounds):
    st = time.time()
    list_agents_model = []
    global_weights = global_model.get_weights()
    avg_weights = []
    model_save = global_model.get_weights()
    for i in global_weights:
      avg_weights.append(np.zeros_like(i))
    # encryptweightlayer(global_weights)
    sum_data_train = 0
    print("Round ", r + 1)
    # Spread Model
    for i in range(agents):
        list_agents_model.append(tf.keras.models.clone_model(global_model))
        list_agents_model[i].set_weights(global_weights)

    # ratio data
    for i in range(agents):
      path = "/content/drive/MyDrive/Khoa_luan/FraudCredit/data-and-scratch/app-data/"
      X_train, Y_train = load_data(path,False, r, i+1)
      sum_data_train += len(Y_train)
    # fed avg
    for i in range(agents):
      path = "/content/drive/MyDrive/Khoa_luan/FraudCredit/data-and-scratch/app-data/"
      X_train, Y_train = load_data(path,False, r, i+1)
      list_agents_model[i].compile(
          loss='categorical_crossentropy',
          optimizer=optimizer,
          metrics=["accuracy"]
      )
      print('Local '+str(i)+' Training:')
      list_agents_model[i].fit(X_train, Y_train, batch_size = 65, epochs=1) #

      local_weights = list_agents_model[i].get_weights()

      ratio = len(Y_train) / sum_data_train

      for j in range(len(local_weights)):
        avg_weights[j] += ratio * (local_weights[j] - model_save[j])

    global_model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=["accuracy"]
    )
    for i in range(len(global_weights)):
      avg_weights[i] = model_save[i] + avg_weights[i]
    global_model.set_weights(avg_weights)

  print('Final Evaluating:')
  [acc,recall,precision,f_score] = Evaluation_Function(global_model, X_test, Y_test)
  return acc,recall,precision,f_score

In [106]:
FL_Train(rounds=10,agents=5,global_model=model)

Round  1
Local 0 Training:
4/4 [==============================] - 1s 11ms/step - loss: 0.1373 - accuracy: 0.9605
Local 1 Training:
4/4 [==============================] - 1s 10ms/step - loss: 0.4188 - accuracy: 0.8070
Local 2 Training:
4/4 [==============================] - 1s 9ms/step - loss: 0.0506 - accuracy: 0.9956
Local 3 Training:
4/4 [==============================] - 1s 11ms/step - loss: 0.4300 - accuracy: 0.7895
Local 4 Training:
4/4 [==============================] - 1s 8ms/step - loss: 0.0469 - accuracy: 0.9912
Round  2
Local 0 Training:
4/4 [==============================] - 1s 7ms/step - loss: 0.1251 - accuracy: 0.9737
Local 1 Training:
4/4 [==============================] - 1s 8ms/step - loss: 0.4441 - accuracy: 0.7939
Local 2 Training:
4/4 [==============================] - 1s 7ms/step - loss: 0.0433 - accuracy: 0.9912
Local 3 Training:
4/4 [==============================] - 1s 6ms/step - loss: 0.4392 - accuracy: 0.8026
Local 4 Training:
4/4 [=============================

(0.9312866, 0.9152047, 0.94561934, 0.9301634351288147)

# TH1

In [ ]:
def FL_Train(rounds,agents,global_model):
  for r in range(rounds):
    list_agents_model = []
    avg_weights = []
    global_weights = global_model.get_weights()
    sum_data_train = 0
    for i in global_weights:
        avg_weights.append(np.zeros_like(i))
    print("Round ", r + 1)
    # Spread Model
    for i in range(agents):
        list_agents_model.append(
            tf.keras.models.clone_model(global_model)
        )
        list_agents_model[i].set_weights(global_weights)
    model_save = global_model.get_weights()
    for i in range(agents):
      path = "/content/drive/MyDrive/Khoa_luan/Fraud_flip/fraud_detection_P2_30/app-data/"
      X_train, Y_train = load_data(path,False, r, i+1)
      list_agents_model[i].compile(
          loss = 'binary_crossentropy',
          optimizer=optimizer,
          metrics=["accuracy"]
      )
      print('Local '+str(i)+' Training:')
      list_agents_model[i].fit(X_train, Y_train, batch_size = 32, epochs= 5,shuffle=False)

      local_weights = list_agents_model[i].get_weights()
      for j in range(len(local_weights)):
        avg_weights[j] += len(Y_train)* (local_weights[j] - model_save[j])
      sum_data_train += len(Y_train)
      for j in range(len(avg_weights)):
          avg_weights[j] = 0.6 * avg_weights[j] / sum_data_train
      global_model = tf.keras.models.clone_model(list_agents_model[0])
      global_model.compile(
              loss = 'binary_crossentropy',
              optimizer=optimizer,
              metrics=["accuracy"]
          )
    for j in range(len(avg_weights)):
        avg_weights[j] = avg_weights[j] + model_save[j]
    global_model.set_weights(avg_weights)
  print('Final Evaluating:')
  acc,recall,precision,f_score = Evaluation_Function(global_model,X_test,Y_test)
  return acc,recall,precision,f_score

In [ ]:
FL_Train(rounds=2,agents=5,global_model=model)

Round  1
Local 0 Training:
Epoch 1/5
8/8 [==============================] - 1s 5ms/step - loss: 0.7627 - accuracy: 0.6579
Epoch 2/5
8/8 [==============================] - 0s 5ms/step - loss: 0.5505 - accuracy: 0.7719
Epoch 3/5
8/8 [==============================] - 0s 4ms/step - loss: 0.4947 - accuracy: 0.7851
Epoch 4/5
8/8 [==============================] - 0s 5ms/step - loss: 0.4602 - accuracy: 0.7895
Epoch 5/5
8/8 [==============================] - 0s 4ms/step - loss: 0.4332 - accuracy: 0.7939
Local 1 Training:
Epoch 1/5
8/8 [==============================] - 1s 5ms/step - loss: 0.5499 - accuracy: 0.8246
Epoch 2/5
8/8 [==============================] - 0s 5ms/step - loss: 0.2377 - accuracy: 0.9430
Epoch 3/5
8/8 [==============================] - 0s 4ms/step - loss: 0.1667 - accuracy: 0.9605
Epoch 4/5
8/8 [==============================] - 0s 5ms/step - loss: 0.1295 - accuracy: 0.9693
Epoch 5/5
8/8 [==============================] - 0s 6ms/step - loss: 0.1064 - accuracy: 0.9868
Local

(0.85672516, 0.86549705, 0.85057473, 0.8579709829770329)

# TH2

In [ ]:
def FL_Train(rounds,agents,global_model):
  for r in range(rounds):
    list_agents_model = []
    avg_weights = []
    global_weights = global_model.get_weights()
    sum_data_train = 0
    for i in global_weights:
        avg_weights.append(np.zeros_like(i))
    print("Round ", r + 1)
    # Spread Model
    for i in range(agents):
        list_agents_model.append(
            tf.keras.models.clone_model(global_model)
        )
        list_agents_model[i].set_weights(global_weights)
    model_save = global_model.get_weights()
    for i in range(agents):
      path = "/content/drive/MyDrive/Khoa_luan/Fraud_flip/fraud_detection_P3_2c_100/app-data/"
      X_train, Y_train = load_data(path,False, r, i+1)
      list_agents_model[i].compile(
          loss='categorical_crossentropy',
          optimizer=optimizer,
          metrics=["accuracy"]
      )
      print('Local '+str(i)+' Training:')
      list_agents_model[i].fit(X_train, Y_train, batch_size = 32, epochs= 5,shuffle=False)

      local_weights = list_agents_model[i].get_weights()
      for j in range(len(local_weights)):
        avg_weights[j] += len(Y_train)* (local_weights[j] - model_save[j])
      sum_data_train += len(Y_train)
      for j in range(len(avg_weights)):
          avg_weights[j] = 0.6 * avg_weights[j] / sum_data_train
      global_model = tf.keras.models.clone_model(list_agents_model[0])
      global_model.compile(
              loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=["accuracy"]
          )
    for j in range(len(avg_weights)):
        avg_weights[j] = avg_weights[j] + model_save[j]
    global_model.set_weights(avg_weights)
  print('Final Evaluating:')
  acc,recall,precision,f_score = Evaluation_Function(global_model,X_test,Y_test)
  return acc,recall,precision,f_score

In [ ]:
FL_Train(rounds=5,agents=5,global_model=model)

Round  1
Local 0 Training:
Epoch 1/5
8/8 [==============================] - 1s 5ms/step - loss: 0.6753 - accuracy: 0.6404
Epoch 2/5
8/8 [==============================] - 0s 5ms/step - loss: 0.5207 - accuracy: 0.7632
Epoch 3/5
8/8 [==============================] - 0s 5ms/step - loss: 0.4658 - accuracy: 0.7763
Epoch 4/5
8/8 [==============================] - 0s 5ms/step - loss: 0.4308 - accuracy: 0.7851
Epoch 5/5
8/8 [==============================] - 0s 5ms/step - loss: 0.4029 - accuracy: 0.7939
Local 1 Training:
Epoch 1/5
8/8 [==============================] - 1s 4ms/step - loss: 0.5674 - accuracy: 0.7237
Epoch 2/5
8/8 [==============================] - 0s 6ms/step - loss: 0.2059 - accuracy: 0.9868
Epoch 3/5
8/8 [==============================] - 0s 5ms/step - loss: 0.1013 - accuracy: 0.9912
Epoch 4/5
8/8 [==============================] - 0s 6ms/step - loss: 0.0627 - accuracy: 0.9912
Epoch 5/5
8/8 [==============================] - 0s 4ms/step - loss: 0.0455 - accuracy: 0.9956
Local

(0.69152045, 0.9590643, 0.6247619, 0.7566320389729361)